### 1. Train the Two Tower network

In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

from utils.utils import *
from utils.utils_train import *
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Suppress huggingface warning

### Load Embeddings

In [2]:
# Load data
queries, query_embeddings, product_embeddings = load_raw_queries()

# load ids_toys from disk
with open('output/ids_toys.pkl', 'rb') as file:
    ids_toys = pickle.load(file)
    print(len(ids_toys))
    ids_present = [_id for _id in ids_toys if _id in query_embeddings]

# Assemble qb & xb input embeddings
qb = torch.stack([torch.tensor(query_embeddings[_id]) for _id in ids_present])
xb = torch.tensor(np.array([product_embeddings[_id] for _id in ids_present]))

### Split Dataset into Train and Test Sets
(qb_train, qb_test), (xb_train, xb_test), (idx_train, idx_test) = shuffle_and_split(qb, xb)
print('qb_train.shape:', qb_train.shape)
print('qb_test.shape:', qb_test.shape)


loaded (queries, query_embeddings, product_embeddings) from  output/raw_queries.pkl
84373
qb_train.shape: torch.Size([20560, 768])
qb_test.shape: torch.Size([5140, 768])


### Two-Tower Pytorch Model

In [3]:
d = hidden_dim = qb.shape[-1]
learning_rate = 0.0001

# Initialize the network
model = TwoTowerNetwork(d, hidden_dim, learning_rate=learning_rate)

list_recall3 = []
list_test_epochs = []


In [ ]:
## TRAIN
import torch
import torch.nn as nn

# Hyperparameters
num_epochs = 10
batch_size = 1024

for i in range(40):
    model.train(qb_train, xb_train, num_epochs=num_epochs, batch_size=batch_size)
    write_output_embeddings(model, qb_test, xb_test) 
    # print(f"epoch no:{model.epochs[-1]:.0f}  ", end='')
    os.system('cd utils && python3 utils_infer.py')
    with open('output/recall3.json', 'r') as file:
        out = json.load(file)
    recall3 = out['recall3']
    list_test_epochs.append(model.epochs[-1])
    list_recall3.append(recall3) 
    

model.plot(qb_train, batch_size)

plt.figure()
plt.plot(list_test_epochs, list_recall3, 'k.-', alpha=.7)
plt.xlim(0)
plt.ylim(0)
plt.title('recall@3')
plt.xlabel('number of epochs');        


recall@3  = 0.022                    
recall@3  = 0.070                    
recall@3  = 0.056                    


### The End

In [3]:
# Load queries, ids_toys, ids_present
with open('output/raw_queries.pkl', 'rb') as file:
    queries, _, _ = pickle.load(file)

with open('output/ids_toys.pkl', 'rb') as file:
    ids_toys = pickle.load(file)
    ids_present = [_id for _id in ids_toys if _id in query_embeddings]


84373


In [5]:
top_ids = [ids_present[idx_test[i]] for i in I[0]]
names = [info[_id]['name'] for _id in top_ids]
details = [info[_id]['details'] for _id in top_ids]
description = [info[_id]['description'] for _id in top_ids]
queries[ids_present[idx_test[0]]]

"I need to buy a Super-duper Tutu/Cape/Mask set in size 4-6 for my 4-6-year-old. It's a 3-piece set perfect for little Superheroines and Rock Stars!"